In [27]:
import os, sys
import numpy as np
import torch
import matplotlib.pyplot as plt
from src2.myModels import end2endModel
import pickle
import csv

In [59]:
# load poses
import pyquaternion as pyq
from scipy.spatial.transform import Rotation as R
import cv2
from src2.Estimate2DOF import * 
from icecream import ic

import importlib

pose_path = '/scratch/aneesh.chavan/KITTI_poses/09.txt'
canon_pose_path = '/scratch/aneesh.chavan/myKITTI_DEMS/09/final_transforms.txt'
dem_path = '/scratch/aneesh.chavan/KITTI/09'

class poseCompare():
    def __init__(self, canon_pose_path=canon_pose_path, pose_path=pose_path, DEM_path=dem_path):
        self.pose_path = pose_path
        self.canon_pose_path = canon_pose_path
        
        # QUATERNIONS ARE SCALAR LAST [x, y, z, w]

        # stored as id and quaternion
        self.canon_pose_dict = {}
        with open(self.canon_pose_path) as f:
            reader = csv.reader(f, delimiter=',')
            ctr = 0
            for row in reader:
                if int(row[0]) > len(self.canon_pose_dict):
                    for key in range(len(self.canon_pose_dict), int(row[0])):
                        self.canon_pose_dict[key] = np.array([0., 0., 0., 0., 0., 0., 1.])
                self.canon_pose_dict[int(row[0])] = np.array([float(i) for i in row[1:]])
                ctr += 1

            #     print(len(self.canon_pose_dict), row[0])

            # print(ctr)
        # stored as P matrices without a quaternion
        self.pose_dict = {}
        with open(self.pose_path) as f:
            reader = csv.reader(f, delimiter=' ')
            for i, row in enumerate(reader):
                t, q = self.P_matrix_to_tq(np.array([float(x) for x in row]))
                self.pose_dict[i] = np.concatenate([t, q])

        self.DEM_path = DEM_path

    # go from pose 1 to pose 2
    # return angle in rads
    """
    Find tx diff and angle diff between two canonicalized pcds and the yaw between them
    needs the rotation of the canonical plane of both pcds (thus the transforms, and intermediate yaws)

    the transform to get from 1 -> 2
        1. apply canonczn tr 1
        2. apply yaw and intermediate translation
        3. apply inv canonczn tr 2
        
    goes from pose 1 to 2
    """
    def get_pred_pose_diff(self, idx1, idx2, viz=False):
        p1 = self.canon_pose_dict[int(idx1)]
        p2 = self.canon_pose_dict[int(idx2)]

        # steps 1 and 3
        Pmatrix1 = self.tq_to_P_matrix(p1)
        Pmatrix2 = self.invert_P_matrix(self.tq_to_P_matrix(p2))

        # ic(p1, p2)

        # step 2, feature matching
        dem1 = cv2.imread(os.path.join(self.DEM_path, ("000" + str(int(idx1)) + ".png")), cv2.IMREAD_GRAYSCALE)
        dem2 = cv2.imread(os.path.join(self.DEM_path, ("000" + str(int(idx2)) + ".png")), cv2.IMREAD_GRAYSCALE)

        # flipped because this function goes from arg 2 to arg 1s
        r, t = DetermineYaw(dem2, dem1, None, None, None, None, viz=viz)

        yaw_matrix = np.eye(4)
        yaw_matrix[:3, :3] = r
        yaw_matrix[:2, -1] = t[:2]

        # yaw_matrix[2,  -1] = distance_r - distance_p # TODO
        yaw_matrix[2,  -1] = 0

        # combine all transforms
        final_transform = Pmatrix2 @ yaw_matrix @ Pmatrix1
        # final_transform = yaw_matrix

        ic(Pmatrix1)
        ic( yaw_matrix)
        ic( Pmatrix2)
        ic(final_transform)

        # get tx and q diff
        tx_diff = np.linalg.norm(final_transform[:3, -1])

        q1 = R.from_matrix(final_transform[:3,:3]).as_quat()
        q2 = np.array([0., 0., 0., 1.])

        angle = np.clip(np.dot(q1, q2), -1.0, 1.0)
        angle = 2*np.arccos(angle)

        return tx_diff, angle, q1, (Pmatrix1, yaw_matrix, Pmatrix2)

    # go from pose 1 to pose 2
    # return angle in rads
    def gt_pose_diff(self, idx1, idx2):
        cn_p1 = self.pose_dict[int(idx1)]
        cn_p2 = self.pose_dict[int(idx2)]

        tx_diff = np.linalg.norm(cn_p2[:3] - cn_p1[:3])

        q1 = cn_p1[3:] / np.linalg.norm(cn_p1[3:])
        q2 = cn_p2[3:] / np.linalg.norm(cn_p2[3:])

        angle = np.clip(np.dot(q1, q2), -1.0, 1.0)
        angle = 2*np.arccos(angle)

        return tx_diff, angle, np.multiply(q2, q1)
    
    def P_matrix_to_tq(self, matrix):
        if len(matrix) == 12:
            matrix = matrix.reshape(3,4)
        t = matrix[:, -1]
        rot_matrix = matrix[:3, :3]

        q = R.from_matrix(rot_matrix).as_quat()

        return t, q

    def tq_to_P_matrix(self, tq):
        t = tq[:3]
        q = tq[3:]

        P = np.eye(4)
        P[:3, -1] = t
        P[:3, :3] = R.from_quat(q).as_matrix()

        return P

    """
    requires a 4x4 matrix
    """
    def invert_P_matrix(self, P):
        assert(len(P) == 4)
        assert(len(P[0]) == 4)

        invP = np.eye(4)

        invR = np.linalg.inv(P[:3, :3])
        try:
            invP[:3, :3] = invR
        except:
            print("Non invertible R matrix (somehow?): ", P[:3, :3])

        invP[:3, -1] = -invR @ P[:3, -1]

        return invP


p = poseCompare(pose_path=pose_path, canon_pose_path=canon_pose_path)

idx1 = 60
idx2 = 61

pred = p.get_pred_pose_diff(idx1,idx2, viz=False)
gt = p.gt_pose_diff(idx1, idx2)

print(pred, gt);

ic| Pmatrix1: array([[ 4.35031489e-01,  9.00398867e-01, -5.42982955e-03,
                      -9.14412623e-01],
                     [-9.00415239e-01,  4.35023579e-01, -2.62339722e-03,
                       4.04783343e-01],
                     [ 4.33680869e-19,  6.03036168e-03,  9.99981817e-01,
                       0.00000000e+00],
                     [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                       1.00000000e+00]])
ic| yaw_matrix: array([[ 9.99999670e-01,  8.12392739e-04,  0.00000000e+00,
                        -2.97737644e-02],
                       [-8.12392739e-04,  9.99999670e-01,  0.00000000e+00,
                         9.45157426e-01],
                       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
                         0.00000000e+00],
                       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                         1.00000000e+00]])
ic| Pmatrix2: array([[ 0.39688081, -0.91787016, -0.        ,  0.73445137],
   

(0.9463583293353691, 0.04128257465619337, array([1.68468509e-03, 8.93878246e-05, 2.05707579e-02, 9.99786976e-01]), (array([[ 4.35031489e-01,  9.00398867e-01, -5.42982955e-03,
        -9.14412623e-01],
       [-9.00415239e-01,  4.35023579e-01, -2.62339722e-03,
         4.04783343e-01],
       [ 4.33680869e-19,  6.03036168e-03,  9.99981817e-01,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]]), array([[ 9.99999670e-01,  8.12392739e-04,  0.00000000e+00,
        -2.97737644e-02],
       [-8.12392739e-04,  9.99999670e-01,  0.00000000e+00,
         9.45157426e-01],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]]), array([[ 0.39688081, -0.91787016, -0.        ,  0.73445137],
       [ 0.91786691,  0.39687941,  0.00266031,  0.67865891],
       [-0.00244182, -0.00105583,  0.99999646, -0.00180545],
       [ 0.      

In [60]:
# save two sample pcds
import open3d as o3d

root_path = '/scratch/aneesh.chavan/KITTI_raw/09/velodyne'
pcd_save_path = '/home2/aneesh.chavan/FinderNetReimplementation/sample_pcds'


def read_velodyne_bin(file_path):
    # Load the binary data
    data = np.fromfile(file_path, dtype=np.float32)
    
    # Reshape the data to be a Nx4 array (x, y, z, intensity)
    points = data.reshape(-1, 4)
    
    # Create an Open3D PointCloud object
    pcd = o3d.geometry.PointCloud()
    
    # Set the points
    pcd.points = o3d.utility.Vector3dVector(points[:, :3])
    
    return pcd


pcd1 = read_velodyne_bin(os.path.join(root_path, f"%.6d.bin" % idx1)).uniform_down_sample(5)
pcd2 = read_velodyne_bin(os.path.join(root_path, f"%.6d.bin" % idx2)).uniform_down_sample(5)

pcd1.paint_uniform_color([1.,0.,0.])
pcd2.paint_uniform_color([0.,0.,1.])

o3d.io.write_point_cloud(os.path.join(pcd_save_path, "anchor.ply"), pcd1)
o3d.io.write_point_cloud(os.path.join(pcd_save_path, "query.ply"), pcd2)
o3d.io.write_point_cloud(os.path.join(pcd_save_path, "both.ply"), pcd1+pcd2)

True

In [61]:
pred = p.get_pred_pose_diff(idx1,idx2, viz=False)
gt = p.gt_pose_diff(idx1, idx2)

pm1 = pred[-1][0]
yawm = pred[-1][1]
pm2 = pred[-1][2]

pred = pred[:-1]

print("GT: ", gt)
print()
print("Pred: ", pred)

print()
print()
ll = R.from_matrix((yawm@pm1)[:3,:3]).as_quat()
ll /= np.linalg.norm(ll)
ll

ic| Pmatrix1: array([[ 4.35031489e-01,  9.00398867e-01, -5.42982955e-03,
                      -9.14412623e-01],
                     [-9.00415239e-01,  4.35023579e-01, -2.62339722e-03,
                       4.04783343e-01],
                     [ 4.33680869e-19,  6.03036168e-03,  9.99981817e-01,
                       0.00000000e+00],
                     [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                       1.00000000e+00]])
ic| yaw_matrix: array([[ 9.99999670e-01,  8.12392739e-04,  0.00000000e+00,
                        -2.97737644e-02],
                       [-8.12392739e-04,  9.99999670e-01,  0.00000000e+00,
                         9.45157426e-01],
                       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
                         0.00000000e+00],
                       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
                         1.00000000e+00]])
ic| Pmatrix2: array([[ 0.39688081, -0.91787016, -0.        ,  0.73445137],
   

GT:  (0.9745814710859236, 0.004499036450207969, array([8.58575488e-04, 6.35818338e-02, 8.12359692e-05, 9.35475825e-01]))

Pred:  (0.9463583293353691, 0.04128257465619337, array([1.68468509e-03, 8.93878246e-05, 2.05707579e-02, 9.99786976e-01]))




array([ 0.00255341, -0.00160359, -0.53183409,  0.8468432 ])

In [62]:
transformed_idx1 = read_velodyne_bin(os.path.join(root_path, f"%.6d.bin" % idx1)).uniform_down_sample(5)

# pm1 = np.array([[ 2.84735335e-01,  9.58599786e-01, -3.49847460e-03,-9.14412623e-01],
#                 [-9.58606170e-01,  2.84733439e-01, -1.03915390e-03,4.04783343e-01],
#                 [-4.33680869e-19,  3.64954317e-03,  9.99993340e-01,0.00000000e+00],
#                 [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,1.00000000e+00]])

# yawm = np.array([[ 2.84735335e-01,  9.58599786e-01, -3.49847460e-03,-9.14412623e-01],
#                  [-9.58606170e-01,  2.84733439e-01, -1.03915390e-03, 4.04783343e-01],
#                  [-4.33680869e-19,  3.64954317e-03,  9.99993340e-01,0.00000000e+00],
#                  [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,1.00000000e+00]])

# pm2 = np.array([[ 0.43503149, -0.90041524, -0.        ,  0.76227138],
#                 [ 0.90039887,  0.43502358,  0.00603036,  0.64724579],
#                 [-0.00542983, -0.0026234 ,  0.99998182, -0.0039032 ],
#                 [ 0.        ,  0.        ,  0.        ,  1.        ]])

t1 = transformed_idx1.transform(pm1)
t2 = t1.transform(yawm)
t3 = t2.transform(pm2)

transformed_idx1 = read_velodyne_bin(os.path.join(root_path, f"%.6d.bin" % idx1)).uniform_down_sample(5)

t4 = transformed_idx1.transform(yawm)

transformed_idx1 = read_velodyne_bin(os.path.join(root_path, f"%.6d.bin" % idx1)).uniform_down_sample(5)

t5 = transformed_idx1.transform(pm1).transform(yawm).transform(pm2)


o3d.io.write_point_cloud(os.path.join(pcd_save_path, "t1.ply"), t1.paint_uniform_color([0.8,0.2,0.]))
o3d.io.write_point_cloud(os.path.join(pcd_save_path, "t2.ply"), t2.paint_uniform_color([0.7,0.3,0.]))
o3d.io.write_point_cloud(os.path.join(pcd_save_path, "t3.ply"), t3.paint_uniform_color([0.6,0.4,0.]))
o3d.io.write_point_cloud(os.path.join(pcd_save_path, "t4.ply"), t4.paint_uniform_color([0.8,0.6,0.]))
o3d.io.write_point_cloud(os.path.join(pcd_save_path, "t5.ply"), t5.paint_uniform_color([0.8,0.6,0.]))


True

In [63]:
r1 = R.from_matrix(

np.array([[ 2.84735335e-01,  9.58599786e-01, -3.49847460e-03,],
          [-9.58606170e-01,  2.84733439e-01, -1.03915390e-03,],
          [-4.33680869e-19,  3.64954317e-03,  9.99993340e-01,]])

).as_euler('xyz')

In [64]:
r2 = R.from_matrix(
    
np.array([[ 0.43503149, -0.90041524, -0.        ],
          [ 0.90039887,  0.43502358,  0.00603036],
          [-0.00542983, -0.0026234 ,  0.99998182]])

).as_euler('xyz')

r2

array([-0.00262344,  0.00542986,  1.12071596])

In [65]:
res = \
np.array([[ 0.43503149, -0.90041524, -0.        ],
          [ 0.90039887,  0.43502358,  0.00603036],
          [-0.00542983, -0.0026234 ,  0.99998182]])  \
@ \
np.array([[ 2.84735335e-01,  9.58599786e-01, -3.49847460e-03,],
          [-9.58606170e-01,  2.84733439e-01, -1.03915390e-03,],
          [-4.33680869e-19,  3.64954317e-03,  9.99993340e-01,]])

print(res)
res = R.from_matrix(res).as_euler('xyz')
res

[[ 9.87012442e-01  1.60642765e-01 -5.86276610e-04]
 [-1.60640914e-01  9.87009932e-01  2.42824081e-03]
 [ 9.68742962e-04 -2.30252676e-03  9.99996882e-01]]


array([-0.00230253, -0.00096874, -0.16134004])

In [66]:
comp = pcd1 + t1 +t2 + t3
o3d.io.write_point_cloud(os.path.join(pcd_save_path, "comp.ply"), comp)


True

In [67]:
def quat_angle_diff(q1, q2):
    q2prime = np.zeros_like(q2)
    q2prime[-1] = q2[-1]
    q2prime[:-1] = q2[:-1]

    angle = np.clip(np.dot(q1, q2prime)/(np.linalg.norm(q1) * np.linalg.norm(q2)), -1.0, 1.0)

    print("q1: ", q1)
    print("q2: ", q2)
    print("angle and arccos: ", angle, np.arccos(angle))

    angle = 2*np.arccos(angle)
    return angle

In [68]:
quat_angle_diff(gt[-1], pred[-1])

q1:  [8.58575488e-04 6.35818338e-02 8.12359692e-05 9.35475825e-01]
q2:  [1.68468509e-03 8.93878246e-05 2.05707579e-02 9.99786976e-01]
angle and arccos:  0.9974946210454558 0.07080149444743467


0.14160298889486933

In [69]:
print(gt[-1],'\n', pred[-1])

angle_diff = quat_angle_diff(gt[-1], pred[-1])
print(angle_diff)

[8.58575488e-04 6.35818338e-02 8.12359692e-05 9.35475825e-01] 
 [1.68468509e-03 8.93878246e-05 2.05707579e-02 9.99786976e-01]
q1:  [8.58575488e-04 6.35818338e-02 8.12359692e-05 9.35475825e-01]
q2:  [1.68468509e-03 8.93878246e-05 2.05707579e-02 9.99786976e-01]
angle and arccos:  0.9974946210454558 0.07080149444743467
0.14160298889486933


In [70]:
np.cos(0.38179632231164684)

0.9279968449248098